In [ ]:
/content/drive/MyDrive/Transformer/Lab-Pages-data.zip

In [ ]:
# !unzip "/content/drive/MyDrive/Transformer/Lab-Pages-data.zip" -d /content/images

In [ ]:
!pip install -q datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from PIL import Image as img
from IPython.display import Image, display

In [ ]:

ds = load_dataset("imagefolder", data_dir="/content/images/Lab-Pages-data")
ds = ds['train']

Resolving data files:   0%|          | 0/401 [00:00<?, ?it/s]

In [ ]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 401
})

In [ ]:
data = ds.train_test_split(test_size=0.15)
data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 340
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 61
    })
})

In [ ]:
data["train"].features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['class_0', 'class_1'], id=None)}

In [ ]:
ex = data['train'][300]
ex

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=2480x3509>,
 'label': 1}

In [ ]:
image = ex['image']
# display(image)

In [ ]:
labels = data['train'].features['label']
labels

ClassLabel(names=['class_0', 'class_1'], id=None)

In [ ]:
labels.int2str(ex['label'])

'class_1'

In [ ]:
labels = data["train"].features["label"].names
print(labels)

['class_0', 'class_1']


In [ ]:
id2label = {k:v for k,v in enumerate(labels)}
label2id = {v:k for k,v in enumerate(labels)}
print(id2label)

{0: 'class_0', 1: 'class_1'}


In [ ]:

id2label[1]

'class_1'

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
from transformers import ConvNextFeatureExtractor

feature_extractor = ConvNextFeatureExtractor.from_pretrained("facebook/convnext-tiny-224")

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/convnext/feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


In [ ]:

print(feature_extractor)


ConvNextFeatureExtractor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConvNextFeatureExtractor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}



In [ ]:
from torchvision.transforms import (
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    ToTensor,
)


In [ ]:

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

transform = Compose(
    [
     RandomResizedCrop(224),
     RandomHorizontalFlip(),
     ToTensor(),
     normalize
    ]
)

def train_transforms(examples):
  examples["pixel_values"] = [transform(image.convert("RGB")) for image in examples["image"]]

  return examples

In [ ]:

data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 340
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 61
    })
})

In [ ]:

processed_dataset = data.with_transform(train_transforms)

In [ ]:

processed_dataset["train"][0]["pixel_values"].shape

torch.Size([3, 224, 224])

In [ ]:
len(labels)

2

In [ ]:

from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("facebook/convnext-tiny-224",
                                                        num_labels=len(labels),
                                                        id2label=id2label,
                                                        label2id=label2id,
                                                        ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/114M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at facebook/convnext-tiny-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

ConvNextForImageClassification(
  (convnext): ConvNextModel(
    (embeddings): ConvNextEmbeddings(
      (patch_embeddings): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (layernorm): ConvNextLayerNorm()
    )
    (encoder): ConvNextEncoder(
      (stages): ModuleList(
        (0): ConvNextStage(
          (downsampling_layer): Identity()
          (layers): Sequential(
            (0): ConvNextLayer(
              (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
              (layernorm): ConvNextLayerNorm()
              (pwconv1): Linear(in_features=96, out_features=384, bias=True)
              (act): GELUActivation()
              (pwconv2): Linear(in_features=384, out_features=96, bias=True)
              (drop_path): Identity()
            )
            (1): ConvNextLayer(
              (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
              (layernorm): ConvNextLayerNorm()
              

In [ ]:
processed_dataset["train"][0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=2480x3507>,
 'label': 1,
 'pixel_values': tensor([[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          ...,
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.1975],
          [2.2489, 2.2489, 2.2489,  ..., 2.1633, 2.1633, 1.5982],
          [2.2489, 2.2489, 2.2489,  ..., 0.6734, 1.0844, 0.8447]],
 
         [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          ...,
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.3761],
          [2.4286, 2.4286, 2.4286,  ..., 2.3410, 2.3410, 1.7633],
          [2.4286, 2.4286, 2.4286,  ..., 0.8179, 1.2381, 0.9930]],
 
         [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
    

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(examples):
  pixel_values = torch.stack([example["pixel_values"] for example in examples])
  labels = torch.tensor([example["label"] for example in examples])

  return {"pixel_values": pixel_values, "labels": labels}

dataloader = DataLoader(processed_dataset["train"], collate_fn=collate_fn, batch_size=4, shuffle=True)

In [ ]:
batch = next(iter(dataloader))
for k,v in batch.items():
  print(k,v.shape)

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


In [ ]:

from tqdm.notebook import tqdm

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(10):
  print("Epoch:", epoch)
  correct = 0
  total = 0
  for idx, batch in enumerate(tqdm(dataloader)):
    # move batch to GPU
    batch = {k:v.to(device) for k,v in batch.items()}

    optimizer.zero_grad()

    # forward pass
    outputs = model(pixel_values=batch["pixel_values"],
                    labels=batch["labels"])

    loss, logits = outputs.loss, outputs.logits
    loss.backward()
    optimizer.step()

    # metrics
    total += batch["labels"].shape[0]
    predicted = logits.argmax(-1)
    correct += (predicted == batch["labels"]).sum().item()

    accuracy = correct/total

    if idx % 100 == 0:
      print(f"Loss after {idx} steps:", loss.item())
      print(f"Accuracy after {idx} steps:", accuracy)

Epoch: 0


  0%|          | 0/85 [00:00<?, ?it/s]

Loss after 0 steps: 0.7253710031509399
Accuracy after 0 steps: 0.5
Epoch: 1


  0%|          | 0/85 [00:00<?, ?it/s]

Loss after 0 steps: 0.016733024269342422
Accuracy after 0 steps: 1.0
Epoch: 2


...

Loss after 0 steps: 0.0415361262857914
Accuracy after 0 steps: 1.0
Epoch: 8


  0%|          | 0/85 [00:00<?, ?it/s]

Loss after 0 steps: 0.0018292003078386188
Accuracy after 0 steps: 1.0
Epoch: 9


  0%|          | 0/85 [00:00<?, ?it/s]

Loss after 0 steps: 0.00080263358540833
Accuracy after 0 steps: 1.0


In [ ]:

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
repo_name = "AnajiK/convnext-tiny-finetuned-LabPages"

model.push_to_hub(repo_name, exist_ok=True)
feature_extractor.push_to_hub(repo_name, exist_ok=True)

In [ ]:

from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("facebook/convnext-tiny-224",

feature_extractor = AutoFeatureExtractor.from_pretrained("nielsr/convnext-tiny-finetuned-eurosat")
model = AutoModelForImageClassification.from_pretrained("nielsr/convnext-tiny-finetuned-eurosat")

In [ ]:

from PIL import Image
import requests

url = " "
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
output_dir = "./Model"
model.save_pretrained(output_dir)
feature_extractor.save_pretrained(output_dir)

['./Model/preprocessor_config.json']

In [ ]:
from PIL import Image as img

In [ ]:
FILE_NAME = '/content/43_class1.png'
image = img.open(FILE_NAME)

pixel_values = feature_extractor(image.convert("RGB"), return_tensors="pt").pixel_values
print(pixel_values.shape)

torch.Size([1, 3, 224, 224])


In [ ]:
import torch

with torch.no_grad():
  outputs = model(pixel_values)
  logits = outputs.logits

print(logits.shape)

predicted_class_idx = logits.argmax(-1).item()
print(model.config.id2label[predicted_class_idx])

In [ ]:

from transformers import pipeline

pipe = pipeline("image-classification",
                model=model,
                feature_extractor=feature_extractor)

In [ ]:

pipe(image)

[{'label': 'class_1', 'score': 0.9989559650421143},
 {'label': 'class_0', 'score': 0.0010440089972689748}]

In [ ]:
# !zip -r Model.zip Model
!unzip -q "/content/drive/MyDrive/Transformer/Model.zip" -d "/content/Models"

Load model

In [ ]:
from PIL import Image as img
import os, time

In [ ]:
from transformers import ConvNextImageProcessor , AutoModelForImageClassification

output_dir = "./Models"
output_dir = "/content/Models/Model"
loaded_tokenizer = ConvNextImageProcessor.from_pretrained(output_dir)
loaded_model = AutoModelForImageClassification.from_pretrained(output_dir)

In [ ]:


FILE_NAME = '/content/40_class0.png'
image = img.open(FILE_NAME)

In [ ]:
from transformers import pipeline
pipe = pipeline("image-classification",
                model=loaded_model,
                feature_extractor=loaded_tokenizer)

In [ ]:
pipe(image)

[{'label': 'class_0', 'score': 0.9913908243179321},
 {'label': 'class_1', 'score': 0.008609114214777946}]

In [ ]:
ls = [i for i in os.listdir('/content/') if i.endswith('.png')]
ls

['40_class0.png',
 '8_class0.png',
 '161_class0.png',
 '62_class1.png',
 '124_class0.png',
 '177_class0.png']

In [ ]:
time.time()

1708664820.8612137

In [ ]:
import tensorflow

In [ ]:
tensorflow.__version__

'2.15.0'

In [ ]:
%%time
for i in ls:
    st = time.time()
    path = os.path.join('/content/', i)
    print(path)
    image = img.open(path)
    rs = pipe(image)
    print(rs)
    max_score = max(rs, key=lambda x: x['score'])
    max_label = max_score['label']
    max_score_value = max_score['score']

    print("prediction Label:", max_label)
    print("score:", max_score_value)
    tm_ = round((time.time()-st),2)
    print("Time: ", tm_)
    print("____"*30)
    print()
    # break

/content/40_class0.png
[{'label': 'class_0', 'score': 0.9913908243179321}, {'label': 'class_1', 'score': 0.008609114214777946}]
prediction Label: class_0
score: 0.9913908243179321
Time:  0.65
________________________________________________________________________________________________________________________

/content/8_class0.png
[{'label': 'class_0', 'score': 0.9718837738037109}, {'label': 'class_1', 'score': 0.02811620570719242}]
prediction Label: class_0
score: 0.9718837738037109
Time:  0.53
________________________________________________________________________________________________________________________

/content/161_class0.png
[{'label': 'class_0', 'score': 0.9997923970222473}, {'label': 'class_1', 'score': 0.0002076041855616495}]
prediction Label: class_0
score: 0.9997923970222473
Time:  0.44
________________________________________________________________________________________________________________________

/content/62_class1.png
[{'label': 'class_1', 'score': 0.99

In [ ]:
torch.__version__

'2.1.0+cu121'

In [ ]:
import torch

# FILE_NAME = '/content/2_class0.png'
image = img.open(FILE_NAME)

pixel_values = loaded_tokenizer(image.convert("RGB"), return_tensors="pt").pixel_values
print(pixel_values.shape)

with torch.no_grad():
  outputs = loaded_model(pixel_values)
  logits = outputs.logits

print(logits.shape)

predicted_class_idx = logits.argmax(-1).item()
print(loaded_model.config.id2label[predicted_class_idx])

torch.Size([1, 3, 224, 224])
torch.Size([1, 2])
class_0


In [ ]:
logits

tensor([[ 2.2250, -2.5213]])

In [ ]:
labels = data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [ ]:
from transformers import ViTFeatureExtractor
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)


In [ ]:

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
feature_extractor.size

{'height': 224, 'width': 224}

In [ ]:
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
            RandomResizedCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(224),
            CenterCrop(224),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [ ]:
# split up training into training + validation
train_ds = data['train']
val_ds = data['test']

In [ ]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [ ]:
from transformers import ViTForImageClassification
model_name_or_path = 'google/vit-base-patch16-224-in21k'
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# !pip install transformers[torch]
!pip install accelerate -U

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'finetuned-indian-food',
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=True,
  report_to='tensorboard',
  load_best_model_at_end=True,
  hub_strategy="end"
)#

In [ ]:
# https://www.youtube.com/watch?v=ahgB8c_TgA8